[View in Colaboratory](https://colab.research.google.com/github/Joovvhan/Master-Thesis/blob/master/src/Transfer_Learning_Clean_Up.ipynb)

In [0]:
# Import necessary modules

import os
import matplotlib.pyplot as plt
import numpy as np
import scipy.io.wavfile as wf
import time

from tqdm import tqdm, trange

In [2]:
# Mount google drive

from google.colab import drive
drive.mount('/content/gdrive')
os.listdir('gdrive/My Drive/Colab')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


['Model', 'Data']

In [0]:
# Set data directories

dataPath = 'gdrive/My Drive/Colab/Data'

# Changed variable names to normal and fault
# Changed variable names from folder to path
# Need to consider multiple folders
# Need to add files to be tested

folderNormal = 'A3F1P3'
folderFault = 'A3F5P3'
pathNormal = dataPath + '/' + folderNormal
pathFault = dataPath + '/' + folderFault
filesNormal = os.listdir(pathNormal)
filesFault = os.listdir(pathFault)

In [0]:
# Specgram settings

nsc = 1470
nov = nsc/2
nff = nsc 
imgSize = 224

In [5]:
# Execution confirmed with new variable names
# Let's add imaging saving features

imgsNormal = np.zeros([len(filesNormal), imgSize, imgSize])

for i in trange(len(filesNormal)):
    fs, dataInt16 = wf.read(pathNormal + '/' + filesNormal[i])
    dataFloat = dataInt16 / (2 ** 15)
    Pxx, _, _, _ = plt.specgram(dataFloat, NFFT=nff, Fs=fs, noverlap=nov, \
                                       window=np.hamming(nsc), cmap='viridis')
    plt.close()
    imgsNormal[i, :, :] = 10 * np.log10(Pxx[0:imgSize, :])

100%|██████████| 1000/1000 [06:28<00:00,  2.94it/s]


In [8]:
# Execution confined with new varible names

imgsFault = np.zeros([len(filesFault), imgSize, imgSize])

for i in trange(len(filesFault)):
    fs, dataInt16 = wf.read(pathFault + '/' + filesFault[i])
    dataFloat = dataInt16 / (2 ** 15)
    Pxx, _, _, _ = plt.specgram(dataFloat, NFFT=nff, Fs=fs, noverlap=nov, \
                                       window=np.hamming(nsc), cmap='viridis')
    plt.close()
    imgsFault[i, :, :] = 10 * np.log10(Pxx[0:imgSize, :])

100%|██████████| 1000/1000 [06:24<00:00,  2.25it/s]


In [0]:
trainIdxF1 = np.random.choice(len(imgsF1) - 1, int(len(imgsF1) * 0.8), replace=False)
testIdxF1 = list(set(range(0, len(imgsF1))) - set(trainIdxF1))

trainImgsF1 = imgsF1[trainIdxF1, :, :]
testImgsF1 = imgsF1[testIdxF1, :, :]

trainIdxF5 = np.random.choice(len(imgsF5) - 1, int(len(imgsF5) * 0.8), replace=False)
testIdxF5 = list(set(range(0, len(imgsF5))) - set(trainIdxF5))

trainImgsF5 = imgsF5[trainIdxF5, :, :]
testImgsF5 = imgsF5[testIdxF5, :, :]

In [0]:
trainImgs = np.vstack([trainImgsF1, trainImgsF5])
testImgs = np.vstack([testImgsF1, testImgsF5])

In [0]:
trainMean = np.mean(trainImgs)
trainStd = np.std(trainImgs)

In [0]:
trainImgsNorm = (trainImgs - trainMean) / trainStd
testImgsNorm = (testImgs - trainMean) / trainStd

trainImgsNorm = trainImgsNorm.reshape(list(trainImgsNorm.shape) + [1])
testImgsNorm = testImgsNorm.reshape(list(testImgsNorm.shape) + [1])

In [0]:
X_train = np.stack([trainImgsNorm[:, :, :, 0], trainImgsNorm[:, :, :, 0], trainImgsNorm[:, :, :, 0]], axis = -1)
X_test = np.stack([testImgsNorm[:, :, :, 0], testImgsNorm[:, :, :, 0], testImgsNorm[:, :, :, 0]], axis = -1)

In [0]:
trainLabelF1 = np.stack((np.ones( int(len(imgsF1) * 0.8)), np.zeros( int(len(imgsF1) * 0.8))), axis = -1)
testLabelF1 = np.stack((np.ones( int(len(imgsF1) * 0.2)), np.zeros( int(len(imgsF1) * 0.2))), axis = -1)

trainLabelF5 = np.stack((np.zeros( int(len(imgsF5) * 0.8)), np.ones( int(len(imgsF5) * 0.8))), axis = -1)
testLabelF5 = np.stack((np.zeros( int(len(imgsF5) * 0.2)), np.ones( int(len(imgsF5) * 0.2))), axis = -1)

Y_train = np.vstack((trainLabelF1, trainLabelF5))
Y_test = np.vstack((testLabelF1, testLabelF5))

In [0]:
from keras.preprocessing import image
from keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.models import Sequential

In [0]:
from keras.applications.inception_v3 import InceptionV3
from keras import backend as K

for i in range(100):

    input_tensor = Input(shape=(224, 224, 3))
    
    modelInceptionV3WoTop = None
    modelInceptionV3 = None

    modelInceptionV3WoTop = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)
    # modelInceptionV3WoTop.summary()
    # modelInceptionV3 = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=True)
    # modelInceptionV3.summary()

    modelInceptionV3 = Sequential()

    modelInceptionV3.add(modelInceptionV3WoTop)
    modelInceptionV3.add(GlobalAveragePooling2D())
    modelInceptionV3.add(Dense(2, activation='softmax'))

    # modelInceptionV3.summary()

    modelInceptionV3.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    modelInceptionV3.fit(X_train, Y_train,
              batch_size=4, epochs=6, verbose=1,
              validation_data=(X_test, Y_test))
    
    K.clear_session()

In [0]:
from keras.applications import VGG19

from keras import backend as K

for i in range(100):
    
    input_tensor = Input(shape=(224, 224, 3))
    
    modelVGG19WoTop = None
    modelVGG19 = None

    modelVGG19WoTop = VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)

    modelVGG19 = Sequential()

    modelVGG19.add(modelVGG19WoTop)
    modelVGG19.add(Flatten())
    modelVGG19.add(Dense(4096, activation='relu'))
    modelVGG19.add(Dropout(0.5))
    modelVGG19.add(Dense(4096, activation='relu'))
    modelVGG19.add(Dropout(0.5))
    
#     modelVGG19.add(Dense(128, activation='relu'))
#     modelVGG19.add(Dropout(0.5))
#     modelVGG19.add(Dense(128, activation='relu'))
#     modelVGG19.add(Dropout(0.5))
    
    modelVGG19.add(Dense(2, activation='softmax'))

    modelVGG19.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    modelVGG19.fit(X_train, Y_train,
              batch_size=2, epochs=8, verbose=1,
              validation_data=(X_test, Y_test))
    
    K.clear_session()

In [0]:
from keras.applications import VGG16

input_tensor = Input(shape=(224, 224, 3))

modelVGG16WoTop = None
modelVGG16 = None

modelVGG16WoTop = VGG16(input_tensor=input_tensor, weights='imagenet', include_top=False)

modelVGG16 = Sequential()

modelVGG16.add(modelVGG16WoTop)
modelVGG16.add(Flatten())
modelVGG16.add(Dense(100, activation='relu'))
modelVGG16.add(Dropout(0.5))
modelVGG16.add(Dense(100, activation='relu'))
modelVGG16.add(Dropout(0.5))

modelVGG16.add(Dense(2, activation='softmax'))

modelVGG16.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

modelVGG16.fit(X_train, Y_train,
          batch_size=2, epochs=2, verbose=1,
          validation_data=(X_test, Y_test))

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions

modelResNet50WoTop = None
modelResNet50 = None

input_tensor = Input(shape=(224, 224, 3))
modelResNet50WoTop = ResNet50(input_tensor=input_tensor, weights='imagenet', include_top=False)

modelResNet50 = Sequential()

modelResNet50.add(modelResNet50WoTop)
modelResNet50.add(Flatten())
modelResNet50.add(Dense(2, activation='softmax'))

modelResNet50.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

modelResNet50.fit(X_train, Y_train,
          batch_size=4, epochs=8, verbose=1,
          validation_data=(X_test, Y_test))

In [0]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import datetime

now = datetime.datetime.now()
modelResNet50.save('gdrive/My Drive/Colab/Model/ResNet50 {}.h5'.format(now.strftime('%m-%d %H:%M:%S')))
# modelResNet50_ = load_model('gdrive/My Drive/Colab/Model/ResNet50.h5')

# Y_pred = modelResNet50_.predict(X_test)
# print(confusion_matrix(np.argmax(Y_pred, axis = 1), np.argmax(Y_test, axis = 1)))

In [0]:
# for i in range(1000, 1010):
#     plt.figure()
#     plt.imshow(X_train[i, :, :, 0]);
#     plt.plot()

